# Dataset Overview

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!apt-get -y install openjdk-8-jre-headless
!pip install pyspark
!pip3 install findspark

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType,StructType,IntegerType,StructField

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
Suggested packages:
  libnss-mdns fonts-dejavu-extra fonts-ipafont-gothic fonts-ipafont-mincho
  fonts-wqy-microhei fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jre-headless
0 upgraded, 1 newly installed, 0 to remove and 45 not upgraded.
Need to get 28.2 MB of archives.
After this operation, 104 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 openjdk-8-jre-headless amd64 8u312-b07-0ubuntu1~18.04 [28.2 MB]
Fetched 28.2 MB in 3s (11.0 MB/s)
Selecting previously unselected package openjdk-8-jre-headless:amd64.
(Reading database ... 155632 files and directories currently installed.)
Preparing to unpack .../openjdk-8-jre-headless_8u312-b07-0ubuntu1~18.04_amd64.

In [ ]:
import pandas as pd
# df = pd.read_csv('public.csv')
df = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/lab03/ccbda_hw5/public.csv')
df

,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,15565701,Ferri,698,Spain,Female,39,9,161993.89,1,0,0,90212.38,0
1,15565706,Akobundu,612,Spain,Male,35,1,0.00,1,1,1,83256.26,1
2,15565796,Docherty,745,Germany,Male,48,10,96048.55,1,1,0,74510.65,0
3,15565806,Toosey,532,France,Male,38,9,0.00,2,0,0,30583.95,0
4,15565878,Bates,631,Spain,Male,29,3,0.00,2,1,1,197963.46,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,15815628,Moysey,711,France,Female,37,8,113899.92,1,0,0,80215.20,0
7996,15815645,Akhtar,481,France,Male,37,8,152303.66,2,1,1,175082.20,0
7997,15815656,Hopkins,541,Germany,Female,39,9,100116.67,1,1,1,199808.10,1
7998,15815660,Mazzi,758,France,Female,34,1,154139.45,1,1,1,60728.89,0


# Use Pyspark to view dataset 

In [ ]:
# These part is  for windows version, if you use ubuntu, remember to edit import pyspark part
# ----
import findspark
findspark.init()
findspark.find()
import pyspark
findspark.find()
# ----
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Churn_Modelling").getOrCreate()
df = spark.read.csv('/content/drive/MyDrive/Colab_Notebooks/lab03/ccbda_hw5/public.csv',header=True,inferSchema=True)
df.printSchema()

root
 |-- CustomerId: integer (nullable = true)
 |-- Surname: string (nullable = true)
 |-- CreditScore: integer (nullable = true)
 |-- Geography: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tenure: integer (nullable = true)
 |-- Balance: double (nullable = true)
 |-- NumOfProducts: integer (nullable = true)
 |-- HasCrCard: integer (nullable = true)
 |-- IsActiveMember: integer (nullable = true)
 |-- EstimatedSalary: double (nullable = true)
 |-- Exited: integer (nullable = true)



# Do your work here

In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml import PipelineModel
from pyspark.ml.feature import RFormula, MinMaxScaler
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LinearSVC, LogisticRegression, RandomForestClassifier

from functools import reduce  # For Python 3.x
from pyspark.sql import DataFrame

from sklearn import metrics
import numpy as np

In [ ]:
## Normalization
def normalize(df):
    print("Normalization...")
    #cols = ["CustomerId", "Surname", "CreditScore", "Geography", "Gender", "Age", "Tenure", "Balance", "NumOfProducts", "HasCrCard", "IsActiveMember", "EstimatedSalary", "Exited"]
    columns_to_scale = ["CreditScore", "Age", "Tenure", "Balance", "NumOfProducts", "HasCrCard", "IsActiveMember", "EstimatedSalary"]

    assemblers = [VectorAssembler(inputCols=[col], outputCol=col + "_vec") for col in columns_to_scale]
    scalers = [MinMaxScaler(inputCol=col + "_vec", outputCol=col + "_scaled") for col in columns_to_scale]

    pipeline = Pipeline(stages=assemblers + scalers)

    scaler_model = pipeline.fit(df)
    scaled_df = scaler_model.transform(df)

    return scaled_df

## Split 5 fold
def split_5_fold(df):
    def unionAll(dfs):
        return reduce(DataFrame.unionAll, dfs)
    
    print("Splitting 5-fold data...")

    folds = df.randomSplit([0.2, 0.2, 0.2, 0.2, 0.2])
    
    print(folds[0].count(), folds[1].count(), folds[2].count(), folds[3].count(), folds[4].count())
    
    train_folds, test_folds = [], []
    for idx in range(len(folds)):
        train = folds[:idx] + folds[idx + 1:]
        train_fold = unionAll(train)
        test_fold = folds[idx]
    
        train_folds.append(train_fold)
        test_folds.append(test_fold)
    
        print("Fold {}: {}, {}".format(idx, train_fold.count(), test_fold.count()))
    
    return train_folds, test_folds

## 5-fold vote
def vote(pred_5_fold):
    pred_5_fold = np.array(pred_5_fold).T.squeeze()
    #print(pred_5_fold)

    votes = []
    for pred in pred_5_fold:
        vote_pred = pred[np.argmax(pred)]
        votes.append(vote_pred)
    
    return votes

In [ ]:
scaled_df = normalize(df)
scaled_df.show()

Normalization...
+----------+--------+-----------+---------+------+---+------+---------+-------------+---------+--------------+---------------+------+---------------+-------+----------+-----------+-----------------+-------------+------------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------------+---------------------+----------------------+
|CustomerId| Surname|CreditScore|Geography|Gender|Age|Tenure|  Balance|NumOfProducts|HasCrCard|IsActiveMember|EstimatedSalary|Exited|CreditScore_vec|Age_vec|Tenure_vec|Balance_vec|NumOfProducts_vec|HasCrCard_vec|IsActiveMember_vec|EstimatedSalary_vec|  CreditScore_scaled|          Age_scaled|       Tenure_scaled|      Balance_scaled|NumOfProducts_scaled|HasCrCard_scaled|IsActiveMember_scaled|EstimatedSalary_scaled|
+----------+--------+-----------+---------+------+---+------+---------+-------------+---------+--------------+---------------+------+------------

In [ ]:
train_folds, test_folds = split_5_fold(scaled_df)

Splitting 5-fold data...
1575 1605 1631 1590 1599
Fold 0: 6425, 1575
Fold 1: 6395, 1605
Fold 2: 6369, 1631
Fold 3: 6410, 1590
Fold 4: 6401, 1599


In [ ]:
## Train on 5-fold data
models = [] ## Save model of each fold
for model_idx in range(2, 3):
    print("-----Model {}-----".format(model_idx))
    for idx in range(len(train_folds)):
        print("Fold {}".format(idx))
        ## Create model
        if model_idx == 0:
            svm = LinearSVC(maxIter=5, regParam=0.01)
            classFormula = RFormula(formula="Exited ~ EstimatedSalary_scaled + IsActiveMember_scaled + HasCrCard_scaled + NumOfProducts_scaled + Balance_scaled + Tenure_scaled + Age_scaled + CreditScore_scaled")
            model = Pipeline(stages=[classFormula, svm]).fit(train_folds[idx])
        elif model_idx == 1:
            lr = LogisticRegression(maxIter=100, regParam=0.3)
            classFormula = RFormula(formula="Exited ~ EstimatedSalary_scaled + IsActiveMember_scaled + HasCrCard_scaled + NumOfProducts_scaled + Balance_scaled + Tenure_scaled + Age_scaled + CreditScore_scaled")
            model = Pipeline(stages=[classFormula, lr]).fit(train_folds[idx])
        elif model_idx == 2:
            rf = RandomForestClassifier(numTrees=100, maxDepth=5)
            classFormula = RFormula(formula="Exited ~ EstimatedSalary_scaled + IsActiveMember_scaled + HasCrCard_scaled + NumOfProducts_scaled + Balance_scaled + Tenure_scaled + Age_scaled + CreditScore_scaled")
            model = Pipeline(stages=[classFormula, rf]).fit(train_folds[idx])

        models.append(model)

-----Model 2-----
Fold 0
Fold 1
Fold 2
Fold 3
Fold 4


In [ ]:
## Test on 5-fold data
for test_fold_idx in range(len(test_folds)):
    print("-----Test on fold {}-----".format(test_fold_idx))
    preds_all = []
    for idx, model in enumerate(models):
        predictions = model.transform(test_folds[test_fold_idx])
        
        gt_array = np.array(predictions.select("label").collect())
        pr_array = np.array(predictions.select("prediction").collect())

        print("Model {}: {}".format(idx, metrics.f1_score(gt_array, pr_array, average='micro')))
        #print(metrics.accuracy_score(gt_array, pr_array))

        preds_all.append(pr_array)
    ## Voting
    votes = vote(preds_all)
    print("Voting : {}".format(metrics.f1_score(gt_array, votes, average='micro')))
    #print(metrics.accuracy_score(gt_array, votes))

-----Test on fold 0-----
Model 0: 0.8533333333333335
Model 1: 0.8596825396825397
Model 2: 0.8590476190476191
Model 3: 0.8584126984126985
Model 4: 0.8603174603174603
Voting : 0.8571428571428571
-----Test on fold 1-----
Model 0: 0.8454828660436137
Model 1: 0.8373831775700935
Model 2: 0.8442367601246106
Model 3: 0.8454828660436137
Model 4: 0.8461059190031153
Voting : 0.8436137071651092
-----Test on fold 2-----
Model 0: 0.8804414469650521
Model 1: 0.8761496014714899
Model 2: 0.8767627222562846
Model 3: 0.882280809319436
Model 4: 0.882280809319436
Voting : 0.8792152053954629
-----Test on fold 3-----
Model 0: 0.8433962264150944
Model 1: 0.8452830188679246
Model 2: 0.8415094339622642
Model 3: 0.8408805031446541
Model 4: 0.8415094339622642
Voting : 0.8427672955974843
-----Test on fold 4-----
Model 0: 0.8636647904940588
Model 1: 0.868042526579112
Model 2: 0.8655409631019386
Model 3: 0.8649155722326454
Model 4: 0.8599124452782989
Voting : 0.8630393996247655


# Public Evaluation Part

## Load private dataset, the same structure as public dataset

In [ ]:
df_private_evl = spark.read.csv('/content/drive/MyDrive/Colab_Notebooks/lab03/ccbda_hw5/public.csv',header=True,inferSchema=True)  # TA takes public dataset as example

## Do prediction with your PySpark model here (**Very Important**)
You must do prediction (inference) here.
Then TA will know how to run your ML model to test private data.

In [ ]:
scaled_df_private = normalize(df_private_evl)

Normalization...


In [ ]:
## Prediction
preds_all = []
for idx, model in enumerate(models):
    predictions = model.transform(scaled_df_private)
    
    gt_array = np.array(predictions.select("label").collect())
    pr_array = np.array(predictions.select("prediction").collect())

    print("Model {}: {}".format(idx, metrics.f1_score(gt_array, pr_array, average='micro')))
    #print(metrics.accuracy_score(gt_array, pr_array))

    preds_all.append(pr_array)
## Voting
votes = vote(preds_all)
print("Voting : {}".format(metrics.f1_score(gt_array, votes, average='micro')))
#print(metrics.accuracy_score(gt_array, votes))

Model 0: 0.857375
Model 1: 0.857375
Model 2: 0.8575
Model 3: 0.8585
Model 4: 0.858125
Voting : 0.85725


## Print Your result as the following type

In [ ]:
df_private_evl.select('CustomerId','Exited').show(5)

+----------+------+
|CustomerId|Exited|
+----------+------+
|  15565701|     0|
|  15565706|     1|
|  15565796|     0|
|  15565806|     0|
|  15565878|     0|
+----------+------+
only showing top 5 rows



## Calculate f-1 score

In [22]:
from sklearn import metrics
import numpy as np
data_array =  np.array(df_private_evl.select('Exited').collect())
data_pred = np.array(votes)
metrics.f1_score(data_array,data_pred)  

0.5461049284578697

In [20]:
print(data_pred)

[0. 0. 1. ... 0. 0. 0.]


# ------------------------------------------------------------------------------------------------------
## Private Evaluation Part (TA will use this block to test your model)
Please 

In [ ]:
df_private = spark.read.csv('private.csv',header=True,inferSchema=True)  # TA takes private dataset as example

## Do prediction with your PySpark model here

## TA will use the following function to get your prediction result (f-1 score)

In [ ]:
from sklearn import metrics
import numpy as np
data_array =  np.array(df_private.select('Exited').collect())

metrics.f1_score(data_array,data_array)  

1.0